# NotPetya Ransomware Activity
Detects NotPetya ransomware activity in which the extracted passwords are passed back to the main module via named pipe, the file system journal of drive C is deleted and windows eventlogs are cleared using wevtutil

## Rule Content
```
- title: NotPetya Ransomware Activity
  id: 79aeeb41-8156-4fac-a0cd-076495ab82a1
  status: experimental
  description: Detects NotPetya ransomware activity in which the extracted passwords
    are passed back to the main module via named pipe, the file system journal of
    drive C is deleted and windows eventlogs are cleared using wevtutil
  author: Florian Roth, Tom Ueltschi
  references:
  - https://securelist.com/schroedingers-petya/78870/
  - https://www.hybrid-analysis.com/sample/64b0b58a2c030c77fdb2b537b2fcc4af432bc55ffb36599a31d418c7c69e94b1?environmentId=100
  tags:
  - attack.execution
  - attack.credential_access
  - attack.defense_evasion
  - attack.t1085
  - attack.t1070
  - attack.t1003
  - car.2016-04-002
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    pipe_com:
      CommandLine: '*\AppData\Local\Temp\\* \\.\pipe\\*'
    rundll32_dash1:
      Image: '*\rundll32.exe'
      CommandLine: '*.dat,#1'
    perfc_keyword:
    - '*\perfc.dat*'
    condition: 1 of them
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Admin activity
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:*\\AppData\\Local\\Temp\\*\ \\.\\pipe\\* OR (process_path.keyword:*\\rundll32.exe AND process_command_line.keyword:*.dat,#1) OR *\\perfc.dat*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()